# Pulse Arrival Analysis

In [ ]:
import os
import numpy as np
from __future__ import (division, absolute_import, print_function)
from scipy import interpolate

from astropy import coordinates as coord
from astropy import units as u
from astropy import constants as const
from astropy import time as astrotime
from astropy.time import Time
from scipy.signal import find_peaks

from  matplotlib import pyplot as plt
import matplotlib

from math import pi

%matplotlib inline

font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

## 1. Reading Data

The ToAs are recorded as a time value in [Modified Julian Date](http://tycho.usno.navy.mil/mjd.html) (MJD). The error is recorded in microseconds.

The barytable has six columns: Firstly the year, month, day number, followed by x, y, z components of the vector in [astronomical units](https://en.wikipedia.org/wiki/Astronomical_unit). This table should be for the year your observations are recorded.

In [ ]:
# Get the data directory path
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir, "Data")

barytable_dir = os.path.join(data_dir, "BaryTables")
toa_dir = os.path.join(data_dir, "TOAs")
period_dir = os.path.join(data_dir, "PeriodEstimates")

save_data_dir = os.path.join(data_dir, "PeriodFitData")

# -----------------------------------------------------
# Functions for reading data
def load_barytable(file_name):
    """
    Load the barytable file.
    
    @param file_name :: name of the file containing barytable data
    
    @returns :: arrays with year, month, day, xpos, ypos, zpos
    """
    
    file_name = file_name + '.asc'
    data_file = os.path.join(barytable_dir, file_name)
    barytable = np.loadtxt(data_file, unpack=True)
    
    return barytable

def load_toas(file_name):
    """
    Load the time of arrivals from file.
    
    @param file_name :: name of the file containing barytable data
    
    @returns :: array containing time of arrival
    """
    
    file_name = file_name + '.asc'
    data_file = os.path.join(toa_dir, file_name)
    toas = np.loadtxt(data_file, usecols=(2,3))
    
    return toas
    
def read_pest(file_name):
    """
    Read the period estimate form file.
    
    @param file_name :: name of the file containing barytable data
    
    @returns :: float with the period estimate
    """
    
    file_name = file_name + '.asc'
    data_file = os.path.join(period_dir, file_name)
    with open(data_file) as dfile:
        line = dfile.read()
        
    period = float(line[23:-8])
    
    return period

## 2. Interpolation of Barycenter


In [ ]:
def compose_date(years, months=1, days=1,
                 hours=None, minutes=None, seconds=None,
                 milliseconds=None):
    """
    Converts array of years, months etc. into numpy64 date array.
    
    @param years :: array of years corresponding to each date
    @param months :: array of months corresponding to each date
    @param days :: array of days corresponding to each date
    @param hours :: array of hours corresponding to each date
    @param minutes :: array of minutes corresponding to each date
    @param seconds :: array of seconds corresponding to each date
    
    @returns :: array of numpy64date format dates
    """
    
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]')
    vals = (years, months, days, hours, minutes, seconds, milliseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)

def mjd_convert(raw_data):
    """
    Converts normal dates from psrchive to modified julian dates.
    
    @param raw_data :: raw array of year, month, day as read from
                       the data file on the barycentre
                       
    @returns :: modified julian date format times
    """
    
    dates = raw_data[:3]
    midnights = np.zeros((4,len(dates[0])))
    times_raw = np.concatenate((dates,midnights))
    times = compose_date(times_raw[0], times_raw[1], times_raw[2],
                         times_raw[3], times_raw[4], times_raw[5],
                         times_raw[6])
    converted_times =  Time(times.astype('string').tolist(), 
                            format='isot', scale='utc')
    mjd_times = converted_times.mjd
    
    return mjd_times

def bary_distance(raw_data):
    """
    Computes the distance from the barycenter to Earth
    on a particular day at a particular time.
    
    @param raw_data :: raw array of year, month, day as read from
                       the data file on the barycentre
                       
    @returns :: magnitude of the x,y,z vector from the raw data
    """
    
    xyz = np.transpose(raw_data[3:])
    dist = np.zeros(len(xyz))
    for i, vect in enumerate(xyz):
        dist[i] += np.sqrt(vect[0]**2 + vect[1]**2 + vect[2]**2)
    
    return dist

In [ ]:
def lag_int(time, order):
    """
    Performs a lagrangian interpolate of order n
    to more accurately determine the Earth-Bary distance at
    a specific time.
    
    @param time :: desired time for which the distance should
                   be calculated
    @param order:: order of the lagrangian interpolate to be calculated
    
    @returns :: the interpolated coordinates and their associated errors
    """
    
    if order == 1:
        raise RuntimeError("Order cannot be 1! Interpolate 1 point? What?")
        
    idxs = np.where(np.logical_and(mjd_times <= time + order/2,
                                   mjd_times >= time - order/2))
    xvals = mjd_times[idxs]
    yvals = distances[idxs]
    interp = interpolate.lagrange(xvals, yvals)
    coords, coords_errors = coord_interp(time, idxs, xvals)
    
    return coords, coords_errors

def coord_interp(time, idxs, xvals):
    """
    Performs a lagrangian interpolate of order n
    to more accurately determine the Earth-Bary coordinate
    at a specific time.
    
    @param time :: desired time for which the distance should
                   be calculated
    @param order :: order of the lagrangian interpolate to be calculated
    
    @returns :: the distance at the specified time and the error on it
    """
    
    coords = []
    coords_errors = []
    for i in range(0, 3):
        yvals = []
        for j in range(idxs[0][0], idxs[0][-1] + 1):
            yvals.append(bary_coord[j][i])
        
        interp = interpolate.lagrange(xvals, yvals)
        coords.append(interp(time))
        coords_errors.append(residual(yvals))
    
    return coords, coords_errors

def residual(yvals):
    """
    Function that finds the delta in determining
    the residuals of the lagrange interpolation formula.
    
    @param yvals :: the Bary-Earth distances of the interpolated
                    points in the data
    
    @returns :: the error due to interpolation on the yvals data
    """
    
    order = len(yvals)
    coeff = [0.125, 0.065, 0.042, 0.031, 
             0.024, 0.019, 0.016]
    delta = yvals
    
    for i in range(0, order-1):        
        delta = np.diff(delta)

    error = coeff[order-2] * delta
    
    return error[0]

## 3. Delays

In [ ]:
# Initialise some constants used throughout
# All these constants are in SI except where specified otherwise
pulsarpos = coord.SkyCoord(ra="05:34:31.93830",
                           dec="22:00:52.1758", 
                           unit=(u.hourangle,u.deg))

lovellpos = coord.EarthLocation.from_geodetic(lat=53.2369, lon=-2.3075)
earth_radius = 6378100.
light_speed = 299792458.

### 3.1 Earth delay

In [ ]:
def earth_delay(times):
    """
    Computes the earth delay in seconds for a set of ToAs.
    
    @param times :: array of ToAs
    
    @returns :: the Earth delay in seconds
    """
    times = astrotime.Time(times, format='mjd')
    altaz = pulsarpos.transform_to(coord.AltAz(obstime=times,
                                               location=lovellpos))
    # Get radians altitude
    alt = coord.Angle(altaz.alt, u.radian)
    earth_delay = earth_radius * np.sin(alt)/light_speed
    
    return earth_delay

### 3.2 Barycentre Delay

In [ ]:
def bary_delay(times):
    """
    Calculates the delay from the center of the Earth to the
    barycenter.
    
    @param time :: time of pulse arrival
    
    @returns :: the delay from center of Earth to Barycenter
    """
    
    pulsarpos = coord.SkyCoord(ra="05:34:31.93830",
                               dec="22:00:52.1758", 
                               unit=(u.hourangle,u.deg))
    pulsarpos.representation = 'cartesian'
    psr_unit = np.array([pulsarpos.x, pulsarpos.y, pulsarpos.z])
    
    delay = []
    for i,time in enumerate(times):
        coords, coords_error = lag_int(time, 3)
        # AU per sec light speed
        light_speed = 0.00200399 
        delay.append(np.dot(coords, psr_unit)/light_speed)

    return delay

## 4. Period Calculation

In [ ]:
def get_remainders(times, period, crab, fileit):
    """
    Function calculates the number of rotations of the star
    
    @param times :: ToAs determined from the file
    @param period :: initial guess for the period of Crab pulsar
    @param crab :: which crab observations you are doing the analysis
                   for, e.g. crab1, crab2 etc.
    
    @return :: saves the remainders of the division between the ToA and
               the time of arrival for every pulse
               also outputs times and corresponding remainders
    """
    
    sec_times = times * 86400
    remainders = sec_times%period
    if fileit == 'y':
        file_data(sec_times, crab + "times.txt")
        file_data(remainders, crab + "remainders.txt")
    
    return times, remainders
    
def file_data(data, filename):
    """
    Function that writes some data to a file, all in a row.
    
    @param data :: 1D array to be saved to file
    @param filename :: the name of the file the data is saved to
    
    @returns :: the written file    
    """
    
    crab_file = os.path.join(save_data_dir, filename)
    with open(crab_file, 'w') as f:
        for elem in data:
            f.write(str(elem) + ' ')

## 4. Analysis

### 4.1 Import data

In [ ]:
# Crab 1 data
crab1_barytable = load_barytable('Crab1Barytable')
crab1_toas = load_toas('Crab1ToAs')
crab1_pest = read_pest('Crab1TEst')

In [ ]:
# Crab 2 data
crab2_barytable = load_barytable('Crab2Barytable')
crab2_toas = load_toas('Crab2ToAs')
crab2_pest = read_pest('Crab2TEst')

In [ ]:
# Calculate mjd times and distances Earth-Bary
# it does not matter what barytable one chooses
# they are all the same

mjd_times = mjd_convert(crab1_barytable)
distances = bary_distance(crab1_barytable)
bary_coord = np.transpose(crab1_barytable[3:])

### 4.2 Compute the Delays

In [ ]:
# First crab observation times
crab1_edelay = earth_delay(crab1_toas[:,0])
crab1_bdelay = bary_delay(crab1_toas[:,0])

barycenter_time1 = crab1_toas[:,0] + (crab1_bdelay + crab1_edelay)/86400

In [ ]:
# Second crab observation times
crab2_edelay = earth_delay(crab2_toas[:,0])
crab2_bdelay = bary_delay(crab2_toas[:,0])

barycenter_time2 = crab2_toas[:,0] + (crab2_bdelay + crab2_edelay)/86400

### 4.3 Remainder Analysis

In [ ]:
# Get times and remainders for first crab observation
period_guess1 = crab1_pest
timesi1, remaindersi1 = \
get_remainders(barycenter_time1, period_guess1, "crab1", 'y')

plt.figure(figsize=(12,7))
plt.scatter(timesi1, remaindersi1)
plt.xlabel('Times [MJD]')
plt.ylabel('Remainders')
plt.title('Crab 1')
plt.show()

In [ ]:
# Crab 1 true period residual plots
true_period1 = 0.03375540288333157

# Alternative period by putting them back to back
true_periodalt1 = 0.03375540747532719
error1 = 5.737285390827286*(10**(-9))

timesf1, remaindersf1 = \
get_remainders(barycenter_time1, true_period1, "crab1", 'n')
timesf1 = np.delete(timesf1, 6)
remaindersf1 = np.delete(remaindersf1, 6)

plt.figure(figsize=(12,7))
plt.scatter(timesf1, remaindersf1)
plt.xlabel('Times [MJD]')
plt.ylabel('Remainders')
plt.title('Crab 1')
plt.show()

In [ ]:
# Get times and remainders for second crab observation
period_guess2 = crab2_pest
timesi2, remaindersi2 = \
get_remainders(barycenter_time2, period_guess2, "crab2", 'y')

plt.figure(figsize=(12,7))
plt.scatter(timesi2, remaindersi2)
plt.xlabel('Times [MJD]')
plt.ylabel('Remainders')
plt.title('Crab 2')
plt.show()

In [ ]:
# Crab 2 true period residual plots
true_period2 = 0.03375569072289549
true_periodalt2 = 0.03375569668845487
error2 = 6.473524733397306**(10^(-9))

timesf2, remaindersf2 = \
get_remainders(barycenter_time2, true_period2, "crab2", 'n')
remaindersf2 = np.delete(remaindersf2, [7,len(remaindersf2)-1])
timesf2 = np.delete(timesf2, [7,len(remaindersf2)-1])

plt.figure(figsize=(12,7))
plt.scatter(timesf2, remaindersf2)
plt.xlabel('Times [MJD]')
plt.ylabel('Remainders')
plt.title('Crab 2')
plt.show()